# Comment LAMMPS DATA FILE


## USING LAMMPS'S FUNCTIONAL FORM AND UNIT

## Read .data file and analyze
* The main goal is to extract the topology and the FF coefficent information
* Before execution, you had better comment the atom types in the "Mass" section of the data file. All information of bond, angle, dihedral, and improper rely on the correct naming of atom type.

In [56]:

from collections import defaultdict

with open("system.data") as file:
    data = file.readlines()

i_Mass = data.index("Masses\n")
i_Atoms = data.index("Atoms  # full\n")
i_Bonds = data.index("Bonds\n")
i_Angles = data.index("Angles\n")
i_Dihedrals = data.index("Dihedrals\n")
i_Impropers = data.index("Impropers\n")

massdata = data[i_Mass+2:i_Atoms-1]
atomdata = data[i_Atoms+2:i_Bonds-1]
bonddata = data[i_Bonds+2:i_Angles-1]
angleldata = data[i_Angles+2:i_Dihedrals-1]
dihedraldata = data[i_Dihedrals+2:i_Impropers-1]
improperdata = data[i_Impropers+2:]

atomtype = dict()
for line in massdata:
    num = line.split()[0]
    # toggle different naming styles, depending on what you need
#     ss = line.split()[5] # unique id for dihedral
#     name = line.split()[3] + "_" + ss[ss.index("d")+1:ss.index("i")-1]
#     name = line.split()[3]+"-"+line.split()[4]
#     name = line.split()[3] + '(' + line.split()[4] + ')'
    name = line.split()[3]
    atomtype[num] = name

atoms = dict()
for line in atomdata:
    num = line.split()[0]
    atype = line.split()[2]
    atoms[num] = atomtype[atype]

#    bonds = dict()
bondtype = defaultdict(list)
for line in bonddata:
    b = line.split()
    if (atoms[b[2]], atoms[b[3]]) not in bondtype[b[1]] and (atoms[b[3]], atoms[b[2]]) not in bondtype[b[1]]:
        bondtype[b[1]].append((atoms[b[2]], atoms[b[3]]))

#    angles = dict()
angletype = defaultdict(list)
for line in angleldata:
    b = line.split()
    if (atoms[b[2]],atoms[b[3]],atoms[b[4]]) not in angletype[b[1]] and (atoms[b[4]],atoms[b[3]],atoms[b[2]]) not in angletype[b[1]]:
        angletype[b[1]].append((atoms[b[2]],atoms[b[3]],atoms[b[4]]))

#    dihedrals = dict()
dihedraltype = defaultdict(list)
for line in dihedraldata:
    b = line.split()
    c = (atoms[b[2]],atoms[b[3]],atoms[b[4]],atoms[b[5]])
    if c not in dihedraltype[b[1]] and c[::-1] not in dihedraltype[b[1]]:
        dihedraltype[b[1]].append(c)

impropertype = defaultdict(list)
for line in improperdata:
    if line == '\n':
        break
    b = line.split()
    c = (atoms[b[2]],atoms[b[3]],atoms[b[4]],atoms[b[5]])
#     if c not in impropertype[b[1]]:
    impropertype[b[1]].append(c)



ValueError: 'Atoms  # full\n' is not in list

**Print FF parameters**

In [55]:
ffname = 'lopls2015' # force field name

with open('pec6.'+ffname+'.settings') as file:
    data_coeff = file.readlines()

    
f = open(ffname+'.ff',"w+")

f.write(ffname + '\n')

f.write("\npair_coeff\n")
for line in data_coeff:
    x = line.split()
    if x[0] == "pair_coeff":
        epislon = float(x[3])
        sigma = float(x[4])
        f.write(f"{x[1]} {sigma:4.2f} {epislon:4.3f} "+atomtype[x[1]]+'\n')

f.write("\nbond_coeff\n")
for line in data_coeff:
    x = line.split()
    if x[0] == "bond_coeff":
        k = float(x[2])
        b = float(x[3])
        f.write(f"{x[1]} {b:4.3f} {k:4.1f} "+'--'.join(list(bondtype[x[1]][0]))+'\n')

f.write("\nangle_coeff\n")
for line in data_coeff:
    x = line.split()
    if x[0] == "angle_coeff":
        k = float(x[2])
        theta0 = float(x[3])
        f.write(f"{x[1]} {theta0:4.1f} {k:4.2f} "+'--'.join(list(angletype[x[1]][0]))+'\n')        
        
f.write("\ndihedral_coeff\n")
for line in data_coeff:
    b = line.split()
    if b[0] == "dihedral_coeff":
        d_para = [float(i) for i in b[2:6]]
        f.write(f"{b[1]} {d_para[0]:7.5f} {d_para[1]:7.5f} {d_para[2]:7.5f} {d_para[3]:7.5f} "+'--'.join(list(dihedraltype[b[1]][0]))+'\n')


f.close()